In [127]:
# Import library yang dibutuhkan
from sklearn.neighbors import KNeighborsClassifier
import cv2
import pickle
import numpy as np
import os
import csv
import time
import pyttsx3
from datetime import datetime
from deepface import DeepFace
from threading import Thread


In [128]:
# Import library yang dibutuhkan
from sklearn.neighbors import KNeighborsClassifier
import cv2
import pickle
import numpy as np
import os
import csv
import time
import pyttsx3
from datetime import datetime
from deepface import DeepFace
from threading import Thread

# Inisialisasi text-to-speech
engine = pyttsx3.init()

# Variable untuk tracking
last_emotion_check = 0
emotion_check_interval = 2.0
current_emotion = "Unknown"
current_emotion_accuracy = 0
face_detection_confidence = 0
last_speech_time = 0
speech_interval = 5.0
is_speaking = False

# Definisi kolom untuk file CSV attendance
COL_NAMES = ['NAME', 'TIME', 'EMOTION', 'EMOTION_ACCURACY', 'FACE_CONFIDENCE']

In [129]:
def speak_async(text):
    global is_speaking
    if not is_speaking:
        is_speaking = True
        engine.say(text)
        engine.runAndWait()
        is_speaking = False


In [130]:
def record_attendance(name, emotion, emotion_acc, face_conf):
    try:
        ts = time.time()
        date = datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp = datetime.fromtimestamp(ts).strftime("%H:%M:%S")
        
        if not os.path.exists("Attendance"):
            os.makedirs("Attendance")
            
        attendance_file = f"Attendance/Attendance_{date}.csv"
        file_exists = os.path.isfile(attendance_file)
        
        # Tambahkan akurasi ke data attendance
        attendance_data = [
            str(name), 
            str(timestamp), 
            str(emotion),
            f"{emotion_acc:.2f}%",
            f"{face_conf:.2f}%"
        ]
        
        with open(attendance_file, "a", newline='') as csvfile:
            writer = csv.writer(csvfile)
            if not file_exists:
                writer.writerow(COL_NAMES)
            writer.writerow(attendance_data)
            
        return True
    except Exception as e:
        print(f"Error recording attendance: {e}")
        return False

In [131]:
def analyze_emotion_async(frame, name):
    global current_emotion, last_speech_time
    try:
        analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        new_emotion = analysis[0]['dominant_emotion']
        
        # Jika emosi berubah dan sudah melewati interval speech
        current_time = time.time()
        if (new_emotion != current_emotion and 
            current_time - last_speech_time > speech_interval):
            
            current_emotion = new_emotion
            # Buat pesan berdasarkan emosi
            if current_emotion == 'happy':
                message = f"Hi {name}, your mood today is happy! Keep up the positive energy!"
            elif current_emotion == 'sad':
                message = f"Hi {name}, your mood today is sad. Stay strong and positive!"
            elif current_emotion == 'angry':
                message = f"Hi {name}, your mood today is angry. Take a deep breath and relax."
            else:
                message = f"Hi {name}, your mood is {current_emotion}. Stay positive!"
            
            # Jalankan speech dalam thread terpisah
            Thread(target=speak_async, args=(message,)).start()
            last_speech_time = current_time
            
    except Exception as e:
        print("Error analyzing emotion:", e)

In [132]:
# Memulai capture video
video = cv2.VideoCapture(0)

# Load classifier dan data
facedetect = cv2.CascadeClassifier(r'C:\Users\ghais\OneDrive\Documents\Purwadhika bootcamp\Modul 3. machine learning\portofolio\Object-Recognition-Project\data\haarcascade_frontalface_default.xml')
with open(r'C:\Users\ghais\OneDrive\Documents\Purwadhika bootcamp\Modul 3. machine learning\portofolio\Object-Recognition-Project\data\names.pkl', 'rb') as f:
    LABELS = pickle.load(f)
with open(r'C:\Users\ghais\OneDrive\Documents\Purwadhika bootcamp\Modul 3. machine learning\portofolio\Object-Recognition-Project\data\faces_data.pkl', 'rb') as f:
    FACES = pickle.load(f)

# Setup KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(FACES, LABELS)

KNeighborsClassifier()

In [133]:
# Variable untuk tracking attendance
last_attendance_name = None
last_attendance_time = 0
attendance_cooldown = 30  # Cooldown 30 detik antara attendance untuk nama yang sama


In [134]:
while True:
    ret, frame = video.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Deteksi wajah dengan confidence
    faces = facedetect.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    
    # Hitung confidence score untuk deteksi wajah
    if len(faces) > 0:
        # Gunakan ukuran wajah terdeteksi sebagai indikator confidence
        x, y, w, h = faces[0]
        face_area = w * h
        frame_area = frame.shape[0] * frame.shape[1]
        face_detection_confidence = min((face_area / frame_area) * 400, 100)  # Scale dan cap at 100%
    
    for (x,y,w,h) in faces:
        crop_img = frame[y:y+h, x:x+w, :]
        resized_img = cv2.resize(crop_img, (50,50)).flatten().reshape(1,-1)
        output = knn.predict(resized_img)
        current_name = str(output[0])
        
        current_time = time.time()
        if current_time - last_emotion_check > emotion_check_interval:
            Thread(target=analyze_emotion_async, args=(frame.copy(), current_name)).start()
            last_emotion_check = current_time
        
        # Draw rectangles and information
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 1)
        cv2.rectangle(frame,(x,y-40),(x+w,y),(50,50,255),-1)
        cv2.putText(frame, current_name, (x,y-15), cv2.FONT_HERSHEY_COMPLEX, 1, (255,255,255), 1)
        
        # Display emotion and accuracies
        y_offset = y + h + 25
        emotion_text = f"Emotion: {current_emotion} ({current_emotion_accuracy:.1f}%)"
        cv2.putText(frame, emotion_text, (x, y_offset), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
        
        face_conf_text = f"Face Detection Confidence: {face_detection_confidence:.1f}%"
        cv2.putText(frame, face_conf_text, (x, y_offset + 25), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
        
        # Add instruction text
        cv2.putText(frame, "Press 'o' to mark attendance", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)
    
    k = cv2.waitKey(1)
    
    # Handle attendance recording
    if k == ord('o') and len(faces) > 0:
        current_time = time.time()
        if (last_attendance_name != current_name or 
            current_time - last_attendance_time > attendance_cooldown):
            
            Thread(target=speak_async, args=("Recording attendance...",)).start()
            
            if record_attendance(current_name, current_emotion, 
                               current_emotion_accuracy, face_detection_confidence):
                last_attendance_name = current_name
                last_attendance_time = current_time
                Thread(target=speak_async, args=("Attendance recorded successfully!",)).start()
            else:
                Thread(target=speak_async, args=("Failed to record attendance.",)).start()
        break
    
    if k == ord('q'):
        break

video.release()
cv2.destroyAllWindows()